In [1]:
import pandas as pd

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


In [3]:
df = pd.read_csv('../data/상장raw.csv')

In [4]:
df['Year']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)

In [14]:
df[(df['거래소코드'] == df['거래소코드'].shift(-1)) & (df['Year'].diff(-1) != -1) &(df['감사의견코드(IFRS)'] ==1)]

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 10차(대분류),부채비율(IFRS연결),당좌비율(IFRS),유동비율(IFRS),이자보상배율(이자비용)(IFRS),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),...,자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원),PER(최고)(IFRS),PER(최저)(IFRS),PBR(최고)(IFRS),PBR(최저)(IFRS),자본금(*)(IFRS연결)(천원),감사의견코드(IFRS),Year
10156,(주)이큐셀,160600,2019/12,29,제조업,382.87,63.03,71.91,-18.41,-18682596.0,...,16974514.0,2232346.0,-29898119.0,0.0,0.0,11.26,1.03,15068102.0,1.0,2019


In [11]:
df['감사의견코드(IFRS)']=df['감사의견코드(IFRS)'].map({
    'AG': 1,
    'DS':1,
    'DI':1,
    'DU':1,
    'QA':0,
    'QC':0,
    'QG':0,
    'QQ':0,
    'QS':1,
    'QU':0,
    'UQ':0,
    'ZZ':1
})

In [12]:
a = df[df['Year'] != 2012]

In [13]:
a[a['회사명'].str.contains('라온텍')]

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 10차(대분류),부채비율(IFRS연결),당좌비율(IFRS),유동비율(IFRS),이자보상배율(이자비용)(IFRS),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),...,자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원),PER(최고)(IFRS),PER(최저)(IFRS),PBR(최고)(IFRS),PBR(최저)(IFRS),자본금(*)(IFRS연결)(천원),감사의견코드(IFRS),Year
2519,(주)라온텍,418420,2013/12,26,제조업,49.47,221.74,322.4,-51.63,-19302.0,...,6452643.0,4265.0,-2468000.0,0.0,0.0,0.0,0.0,2175000.0,NaN,2013
2520,(주)라온텍,418420,2021/12,26,제조업,NaN,NaN,NaN,NaN,591946.0,...,5791910.0,2701154.0,-1564126.0,NaN,NaN,NaN,NaN,2496175.0,0.0,2021
2521,(주)라온텍,418420,2022/12,26,제조업,68.26,372.51,483.3,8.65,404090.0,...,7482903.0,4234868.0,322445.0,0.0,0.0,0.0,0.0,2496175.0,0.0,2022


In [16]:
df_외감 = pd.read_csv('../data/외감raw.csv', encoding='euc-kr')

In [22]:
df_외감['감사의견코드']=df_외감['감사의견코드'].map({
    'AG': 1,
    'DS':1,
    'DI':1,
    'DU':1,
    'QA':0,
    'QC':0,
    'QG':0,
    'QQ':0,
    'QS':1,
    'QU':0,
    'UQ':0,
    'ZZ':1
})

In [18]:
# '거래소코드'로 그룹화하여 처리
grouped = df_외감.groupby('거래소코드')

df_외감['새로운 인덱스'] = grouped.cumcount() + 1

# 처리할 새로운 DataFrame을 저장할 리스트 초기화
result_data = []

for group_key, group_df_외감 in grouped:
    # '감사의견코드' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_df_외감[group_df_외감['감사의견코드'] == 1].index[0]
    
    if group_df_외감.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_df_외감.drop(index=group_df_외감.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_df_외감.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_df_외감)

# 처리 결과를 하나의 DataFrame으로 병합
result_df_외감 = pd.concat(result_data)

result_df_외감


IndexError: index 0 is out of bounds for axis 0 with size 0

In [23]:
import pandas as pd

# 거래소코드로 그룹화하여 처리하는 함수
def process_group(group):
    # 빈 그룹인 경우 그대로 반환
    if group.empty:
        return group
    
    # 거래소코드 그룹별로 처리
    if 1 in group['감사의견코드'].values:
        # 1이 처음으로 나오는 인덱스 찾기
        first_index = group.index[group['감사의견코드'] == 1][0]
        
        # 처음으로 1이 나오는 행의 아래 행들 삭제
        group = group.loc[:first_index]
    else:
        # 1이 없는 경우 그룹 그대로 유지
        pass
    
    return group

# '거래소코드'로 그룹화하여 함수를 적용하고 다시 합치기
processed_df = df_외감.groupby('거래소코드').apply(process_group)

processed_df


회사명   거래소코드     회계년도  통계청 한국표준산업분류 코드 10차(대분류)    부채비율  \
거래소코드                                                                         
10000  269007   (주)포스코건설   10000  2012/12                        41  173.49   
       269008   (주)포스코건설   10000  2013/12                        41  130.89   
       269009   (주)포스코건설   10000  2014/12                        41  120.92   
       269010   (주)포스코건설   10000  2015/12                        41  100.59   
       269011   (주)포스코건설   10000  2016/12                        41  145.02   
...                  ...     ...      ...                       ...     ...   
107274 215212  주식회사이안이앤디  107274  2022/12                        68    0.00   
107276 216611    주식회사일그램  107276  2021/12                        71   99.28   
       216612    주식회사일그램  107276  2022/12                        71  145.67   
107278 32679   (주)두산종합건재  107278  2021/12                        46  273.78   
       32680   (주)두산종합건재  107278  2022/12                        46  909.68   

                  당좌비율     유동비율  이자보상배율(이자비용)  \
거래소코드                                           
10000  269007   124.35   149.99         14.79   
       269008   120.59   146.23         13.84   
       269009   125.34   153.62          6.33   
       269010   145.66   164.20          7.33   
       269011   109.23   133.86         -8.08   
...                ...      ...           ...   
107274 215212    11.04    79.11         -0.71   
107276 216611  1838.90  1838.90         25.74   
       216612   448.46   448.46         13.22   
107278 32679    376.36   394.26          1.13   
       32680    284.24   291.35         -6.44   

               영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)  이자비용(IFRS연결)(천원)  ...  \
거래소코드                                                                ...   
10000  269007                         123580817.0        45854000.0  ...   
       269008                         383458439.0        44520000.0  ...   
       269009                         -81029724.0        55648000.0  ...   
       269010                         645297841.0        61387000.0  ...   
       269011                        -616838633.0        45847000.0  ...   
...                                           ...               ...  ...   
107274 215212                         -20227789.0         1314616.0  ...   
107276 216611                           3232717.0          108993.0  ...   
       216612                           3770208.0          315760.0  ...   
107278 32679                           -2107626.0          241431.0  ...   
       32680                             154239.0          328282.0  ...   

               투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)  재무활동으로 인한 현금흐름(*)(IFRS연결)(천원)  \
거래소코드                                                                         
10000  269007                   -331194515.0                    121474510.0   
       269008                   -150352822.0                   -163581203.0   
       269009                   -527581873.0                    -27739975.0   
       269010                      5879907.0                    391500030.0   
       269011                   -244628460.0                    146163668.0   
...                                      ...                            ...   
107274 215212                      -523459.0                     20739181.0   
107276 216611                     -9688828.0                      6879900.0   
       216612                    -10180053.0                      5830000.0   
107278 32679                       -825371.0                      2865000.0   
       32680                       -192254.0                        79100.0   

               자산(*)(IFRS연결)(천원)  부채(*)(IFRS연결)(천원)  자본(*)(IFRS연결)(천원)  \
거래소코드                                                                    
10000  269007       9.200763e+09       6.371979e+09       2.828785e+09   
       269008       8.641621e+09       5.695142e+09       2.946479e+09   
       269009       8.366265e+09       5.3

In [24]:
# processed_df.to_csv('test.csv', encoding='euc-kr')

In [26]:
processed_df['감사의견코드'].value_counts()

0.0    208286
1.0     15598
Name: 감사의견코드, dtype: int64

In [29]:
# 처리할 새로운 DataFrame을 저장할 리스트 초기화
result_data = []

for group_key, group_processed_df in grouped:
    # 'target' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_processed_df[group_processed_df['감사의견코드'] == 1].index[0]
    
    if group_processed_df.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_processed_df.drop(index=group_processed_df.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_processed_df.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_processed_df)

# 처리 결과를 하나의 DataFrame으로 병합
result_processed_df = pd.concat(result_data)

result_processed_df


IndexError: index 0 is out of bounds for axis 0 with size 0

In [30]:
result_data = []

for group_key, group_processed_df in grouped:
    # 그룹화된 데이터프레임이 비어있는 경우, 처리를 건너뛰고 다음 그룹으로 넘어감
    if group_processed_df.empty:
        continue
    
    # 'target' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_processed_df[group_processed_df['감사의견코드'] == 1].index[0]
    
    if group_processed_df.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_processed_df.drop(index=group_processed_df.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_processed_df.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_processed_df)

# 처리 결과를 하나의 DataFrame으로 병합
result_processed_df = pd.concat(result_data)

result_processed_df

IndexError: index 0 is out of bounds for axis 0 with size 0

---

In [26]:
def normality_test(df, tool='Shapiro'):

    from scipy.stats import shapiro, anderson, kstest,jarque_bera, normaltest,anderson_ksamp

    result_dict = {'col':[], 'p_value':[], 't_stat':[]}

    if tool == 'Shapiro':
        for i in df.columns:
            statistic, p_value = shapiro(df[i])
            result_dict['col'].append(i)
            result_dict['p_value'].append(p_value)
            result_dict['t_stat'].append(statistic)

            test = pd.DataFrame(result_dict)
    
    elif tool == 'k_s':
        for i in df.columns:
            statistic, p_value = kstest(df[i], 'norm')
            result_dict['col'].append(i)
            result_dict['p_value'].append(p_value)
            result_dict['t_stat'].append(statistic)

            test = pd.DataFrame(result_dict)

    elif tool == 'jarque_bera':
        for i in df.columns:
            statistic, p_value = jarque_bera(df[i])
            result_dict['col'].append(i)
            result_dict['p_value'].append(p_value)
            result_dict['t_stat'].append(statistic)

            test = pd.DataFrame(result_dict)


    elif tool == 'anderson':

        result_dict = {'col' : [], 'critical_values' : [], 't_stat' : []}

        for i in df.columns:
            result = anderson(df[i])
            result_dict['col'].append(i)
            result_dict['critical_values'].append(result.critical_values[2])
            result_dict['t_stat'].append(result.statistic)

            test = pd.DataFrame(result_dict)

    return test

In [116]:
df = pd.read_csv('../data/dfdf.csv', encoding='euc-kr')

In [75]:
import numpy as np
def trimmed_mean_with_ratio(data, left_trim_ratio=0.01, right_trim_ratio=0.01):
    data.sort()
    left_trim_size = int(len(data) * left_trim_ratio)
    right_trim_size = int(len(data) * right_trim_ratio)
    trimmed_data = data[left_trim_size:-right_trim_size]


In [87]:
df_1 = df[df['t-1감사의견코드']==1]

In [88]:
df_0 = df[df['t-1감사의견코드']==0]

In [89]:
df_0 = df_0[['매출액증가율','부채비율','이자보상배율','유동자산증가율','자기자본구성비율','자기자본증가율','재고자산증가율','차입금의존도','총자본증가율']]

In [90]:
df_1 = df_1[['매출액증가율','부채비율','이자보상배율','유동자산증가율','자기자본구성비율','자기자본증가율','재고자산증가율','차입금의존도','총자본증가율']]

In [91]:
trimmed_mean_with_ratio(df_0['매출액증가율'].values)
trimmed_mean_with_ratio(df_0['부채비율'].values, left_trim_ratio=0.03, right_trim_ratio=0.03)
trimmed_mean_with_ratio(df_0['이자보상배율'].values)
trimmed_mean_with_ratio(df_0['유동자산증가율'].values)
trimmed_mean_with_ratio(df_0['자기자본구성비율'].values)
trimmed_mean_with_ratio(df_0['자기자본증가율'].values)
trimmed_mean_with_ratio(df_0['재고자산증가율'].values)
trimmed_mean_with_ratio(df_0['차입금의존도'].values)
trimmed_mean_with_ratio(df_0['총자본증가율'].values)

In [92]:
trimmed_mean_with_ratio(df_1['매출액증가율'].values)
trimmed_mean_with_ratio(df_1['부채비율'].values)
trimmed_mean_with_ratio(df_1['이자보상배율'].values)
trimmed_mean_with_ratio(df_1['유동자산증가율'].values)
trimmed_mean_with_ratio(df_1['자기자본구성비율'].values)
trimmed_mean_with_ratio(df_1['자기자본증가율'].values)
trimmed_mean_with_ratio(df_1['재고자산증가율'].values)
trimmed_mean_with_ratio(df_1['차입금의존도'].values)
trimmed_mean_with_ratio(df_1['총자본증가율'].values)

In [93]:
df_0.describe()

,매출액증가율,부채비율,이자보상배율,유동자산증가율,자기자본구성비율,자기자본증가율,재고자산증가율,차입금의존도,총자본증가율
count,1.726240e+05,1.726240e+05,1.726240e+05,1.726240e+05,172624.000000,1.726240e+05,1.726240e+05,172624.00000,172624.000000
mean,7.768561e+02,1.672486e+04,1.161928e+08,5.837017e+01,39.754937,4.872890e+01,2.388049e+02,38.21257,8.698038
std,1.363954e+05,5.732536e+06,3.204441e+08,4.207398e+03,27.371267,4.006669e+03,2.633683e+04,35.35415,61.610195
min,-1.813100e+02,0.000000e+00,-3.620049e+07,-9.999000e+01,-22.870000,-1.299710e+03,-1.000000e+02,0.00000,-99.070000
25%,-6.920000e+00,4.504000e+01,6.200000e-01,-8.230000e+00,17.430000,0.000000e+00,-4.460000e+00,9.07000,-2.980000
50%,0.000000e+00,1.355600e+02,2.810000e+00,1.690000e+00,36.790000,2.710000e+00,0.000000e+00,35.59000,0.850000
75%,1.435000e+01,3.263425e+02,2.257000e+01,2.041000e+01,60.450000,1.368000e+01,1.025000e+01,58.98000,11.310000
max,5.198966e+07,2.373601e+09,1.733808e+09,1.592057e+06,100.000000,1.562930e+06,8.715313e+06,2723.79000,11695.370000


In [94]:
df_1.describe()

,매출액증가율,부채비율,이자보상배율,유동자산증가율,자기자본구성비율,자기자본증가율,재고자산증가율,차입금의존도,총자본증가율
count,12470.000000,1.247000e+04,1.247000e+04,1.247000e+04,12470.000000,12470.000000,12470.000000,12470.000000,12470.000000
mean,99.963354,2.470937e+03,7.402333e+07,4.331525e+02,26.530341,15.050464,82.190784,58.477393,1.542986
std,6623.782262,6.174527e+04,2.618086e+08,4.729948e+04,23.432919,1138.687063,8380.575608,916.713208,31.834279
min,-99.980000,0.000000e+00,-1.033049e+06,-1.000000e+02,-2.160000,-99.900000,-99.870000,0.000000,-100.000000
25%,0.000000,7.111750e+01,1.100000e-01,0.000000e+00,7.760000,0.000000,0.000000,25.905000,0.000000
50%,0.000000,2.498800e+02,2.660000e+00,0.000000e+00,21.340000,0.000000,0.000000,51.800000,0.000000
75%,0.000000,5.573000e+02,1.539500e+01,0.000000e+00,39.047500,0.000000,0.000000,69.607500,0.000000
max,727175.260000,5.353429e+06,1.000000e+09,5.281818e+06,99.850000,126156.120000,935625.220000,102094.630000,2015.270000


In [99]:
df= df[['매출액증가율','부채비율','이자보상배율','유동자산증가율','자기자본구성비율','자기자본증가율','재고자산증가율','차입금의존도','총자본증가율','t-1감사의견코드']]

In [104]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.

X_train, X_test, y_train, y_test = train_test_split(df.drop('t-1감사의견코드', axis=1), df['t-1감사의견코드'], stratify=df['t-1감사의견코드'])

In [100]:
trimmed_mean_with_ratio(X_train['매출액증가율'].values)
trimmed_mean_with_ratio(X_train['부채비율'].values, left_trim_ratio=0.03, right_trim_ratio=0.03)
trimmed_mean_with_ratio(X_train['이자보상배율'].values)
trimmed_mean_with_ratio(X_train['유동자산증가율'].values)
trimmed_mean_with_ratio(X_train['자기자본구성비율'].values)
trimmed_mean_with_ratio(X_train['자기자본증가율'].values)
trimmed_mean_with_ratio(X_train['재고자산증가율'].values)
trimmed_mean_with_ratio(X_train['차입금의존도'].values)
trimmed_mean_with_ratio(X_train['총자본증가율'].values)

In [107]:
from sklearn.metrics import f1_score

In [105]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [108]:
# 예측
y_pred = tree.predict(X_test)

# F1 score 계산
f1 = f1_score(y_test, y_pred)
print('F1 score:', f1)

F1 score: 0.32002764817694834


 # 정규성검정

In [48]:
normality_test(df_0)

,col,p_value,t_stat
0,매출액증가율,1.0,NaN
1,부채비율,1.0,NaN
2,이자보상배율,1.0,NaN
3,유동자산증가율,1.0,NaN
4,자기자본구성비율,1.0,NaN
5,자기자본증가율,1.0,NaN
6,재고자산증가율,1.0,NaN
7,차입금의존도,1.0,NaN
8,총자본증가율,1.0,NaN


In [49]:
normality_test(df_1)

,col,p_value,t_stat
0,매출액증가율,1.0,NaN
1,부채비율,1.0,NaN
2,이자보상배율,1.0,NaN
3,유동자산증가율,1.0,NaN
4,자기자본구성비율,1.0,NaN
5,자기자본증가율,1.0,NaN
6,재고자산증가율,1.0,NaN
7,차입금의존도,1.0,NaN
8,총자본증가율,1.0,NaN


# 스케일링 데이터 정규성검정

In [29]:
from sklearn.preprocessing import StandardScaler

In [7]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '산업분류코드', '부채비율', '당좌비율', '유동비율', '이자보상배율',
       'OCF(천원)', '이자비용(천원)', '차입금의존도', '자기자본구성비율', '당기순이익(천원)', '감가상각비(백만원)',
       '금융비용(백만원)', '단기차입금(천원)', '유동성장기부채(천원)', '장기차입금(천원)', '법인세비용차감전손익(백만원)',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '매출액(천원)', '총자본회전률', '자기자본회전률',
       '운전자본회전률', '순운전자본회전률', '재고자산회전률', '당좌자산회전률', '유동자산회전률', '매출액증가율',
       '총자본증가율', '자기자본증가율', '순이익증가율', '유형자산증가율', '유동자산증가율', '재고자산증가율',
       '영업이익증가율', '총자본투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', 'ICF(천원)',
       'FCF(천원)', '자산총계(천원)', '부채총계(천원)', '자본총계(천원)', '산업분류', '자본금(천원)',
       '영업손익(천원)', '연구개발비(천원)', '자본잠식률', 'Year', 't-1감사의견코드'],
      dtype='object')

In [39]:
sc = StandardScaler()
df_0 = sc.fit_transform(df_0)
df_1 = sc.fit_transform(df_1)

In [56]:
from scipy.stats import shapiro

# Shapiro-Wilk 검정 함수 정의
def shapiro_test(data):
    stat, p = shapiro(data)
    return stat, p

# Shapiro-Wilk 검정 수행
stat_0, p_0 = shapiro_test(df_0)
stat_1, p_1 = shapiro_test(df_1)

# 결과 출력
print("Shapiro-Wilk 검정 결과 (df_0):")
print("Statistic:", stat_0)
print("p-value:", p_0)
if p_0 > 0.05:
    print("p-value > 0.05 이므로 정규성을 만족합니다.")
else:
    print("p-value <= 0.05 이므로 정규성을 만족하지 않습니다.")

print("\nShapiro-Wilk 검정 결과 (df_1):")
print("Statistic:", stat_1)
print("p-value:", p_1)
if p_1 > 0.05:
    print("p-value > 0.05 이므로 정규성을 만족합니다.")
else:
    print("p-value <= 0.05 이므로 정규성을 만족하지 않습니다.")


Shapiro-Wilk 검정 결과 (df_0):
Statistic: nan
p-value: 1.0
p-value > 0.05 이므로 정규성을 만족합니다.

Shapiro-Wilk 검정 결과 (df_1):
Statistic: nan
p-value: 1.0
p-value > 0.05 이므로 정규성을 만족합니다.


---

In [113]:
import pandas as pd

,(코드)산업,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
0,(0001)전산업,7.19,8.44,11.09,7.41,7.40,6.74,8.71,8.90,10.27,5.97
1,(0002)농림어업,3.19,7.29,13.01,14.53,13.05,11.48,9.09,-1.01,17.06,0.26
2,(0003)광업,6.9,-4.76,7.79,11.68,10.60,10.86,9.50,2.20,37.22,3.29
3,(0004)제조업,6.37,5.19,4.47,5.78,7.15,5.25,4.54,5.92,11.15,9.28
4,(0005)전기가스수도,6.87,7.14,3.87,1.70,2.69,3.00,5.14,1.51,7.03,15.81
...,...,...,...,...,...,...,...,...,...,...,...
80,주기,[년],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,탑재기간,1994~2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,업데이트 일자,2023-07-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,비고,ㅇ 원출처 : KISVALUEㅇ 최근연도 외감기업 기준,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df['산업분류'].value_counts()

제조업                         75978
부동산업                        25235
도매 및 소매업                    22221
건설업                         17637
운수 및 창고업                     8831
정보통신업                        7797
전문, 과학 및 기술 서비스업             6156
사업시설 관리, 사업 지원 및 임대 서비스업     5612
숙박 및 음식점업                    3517
예술, 스포츠 및 여가관련 서비스업          3390
수도, 하수 및 폐기물 처리, 원료 재생업      2333
전기, 가스, 증기 및 공기조절 공급업        2040
협회 및 단체, 수리 및 기타 개인 서비스업     1326
농업, 임업 및 어업                   825
금융 및 보험업                      811
교육 서비스업                       727
광업                            536
보건업 및 사회복지 서비스업               111
공공 행정, 국방 및 사회보장 행정            11
Name: 산업분류, dtype: int64

---

In [ ]:
부동산업 = {
    2013: 289.88,
    2014: 321.49,
    2015: 337.6,
    2016: 329.55,
    2017: 318.05,
    2018: 321.98,
    2019: 352.7,
    2020: 386.86,
    2021: 420.31,
    2022: 413.37
}

In [ ]:
도매및소매업 = {
    2013: 114.9,
    2014: 108.09,
    2015: 106.03,
    2016: 104.37,
    2017: 99.51,
    2018: 100.44,
    2019: 108.46,
    2020: 113.86,
    2021: 116.12,
    2022: 117.37
}

In [ ]:
건설업 = {
    2013: 165.76,
    2014: 163.29,
    2015: 169.4,
    2016: 160.23,
    2017: 140.63,
    2018: 128.14,
    2019: 130.63,
    2020: 128.05,
    2021: 129.54,
    2022: 140.01
}

In [ ]:
운수및창고업 = {
    2013: 403.4,
    2014: 351.48,
    2015: 350.91,
    2016: 325.06,
    2017: 294.12,
    2018: 300.4,
    2019: 343.29,
    2020: 354.74,
    2021: 238.84,
    2022: 171.36
}

In [ ]:
정보통신업 = {
    2013: 86.11,
    2014: 80.835,
    2015: 76.4,
    2016: 77.01,
    2017: 71.62,
    2018: 68.38,
    2019: 77.16,
    2020: 82.135,
    2021: 97.25,
    2022: 92.285
}

In [ ]:
전문과학기술 = {
    2013: 64.59,
    2014: 59.77,
    2015: 54.17,
    2016: 52.1,
    2017: 54.67,
    2018: 52.32,
    2019: 53.42,
    2020: 56.42,
    2021: 51.48,
    2022: 54.48
}

In [ ]:
사업지원및사업시설관리서비스업 = {
    2013: 332,
    2014: 347.875,
    2015: 344.75,
    2016: 343.275,
    2017: 351,
    2018: 321.05,
    2019: 303.74,
    2020: 270.4,
    2021: 288.89,
    2022: 267.6
}

In [ ]:
사업지원및사업시설관리서비스업 = {
    2013: 332,
    2014: 347.875,
    2015: 344.75,
    2016: 343.275,
    2017: 351,
    2018: 321.05,
    2019: 303.74,
    2020: 270.4,
    2021: 288.89,
    2022: 267.6
}

In [ ]:
숙박및음식점업 = {
    2013: 101.96,
    2014: 109.1,
    2015: 120.84,
    2016: 121.85,
    2017: 135.44,
    2018: 139.5,
    2019: 162.26,
    2020: 175.45,
    2021: 195.49,
    2022: 178.17
}

In [ ]:
예술스포츠및여가관련서비스업 = {
    2013: 345.34,
    2014: 289.14,
    2015: 248.01,
    2016: 197.38,
    2017: 188.18,
    2018: 163.42,
    2019: 160.96,
    2020: 161.63,
    2021: 159,
    2022: 143.32
}

In [ ]:
수도하수및폐기물처리원료재생업 = {
    2013: 190.85,
    2014: 194.48,
    2015: 192.13,
    2016: 180.26,
    2017: 168.91,
    2018: 155.07,
    2019: 155.16,
    2020: 155.03,
    2021: 138.85,
    2022: 130.29
}

In [ ]:
전기가스증기및공기조절공급업 = {
    2013: 156.44,
    2014: 159.33,
    2015: 137.91,
    2016: 131.77,
    2017: 133.93,
    2018: 141.99,
    2019: 159.89,
    2020: 159.08,
    2021: 181.3,
    2022: 272.92
}

In [ ]:
보건업및사회복지서비스업 = {
    2013: 543.28,
    2014: 569.05,
    2015: 572.8,
    2016: 509.1,
    2017: 389.89,
    2018: 424.205,
    2019: 383.98,
    2020: 447.81,
    2021: 152.9,
    2022: 124.98
}

In [2]:
df = pd.read_csv('../data/t-1_raw_외감.csv',encoding='euc-kr')

In [7]:
df[(df['이자보상배율']==0)&(df['t-1감사의견코드']==1)]

,회사명,거래소코드,회계년도,산업분류코드,부채비율,당좌비율,유동비율,이자보상배율,OCF(천원),이자비용(천원),...,자산총계(천원),부채총계(천원),자본총계(천원),산업분류,자본금(천원),영업손익(천원),연구개발비(천원),자본잠식률,Year,t-1감사의견코드
68931,목포조선공업(주),49024,2013/12,31,0.00,59.47,60.13,0.0,330933.0,1095748.0,...,12046892.0,16978031.0,-4931139.0,제조업,111910.0,-2724.0,0.0,45.063435,2013,1.0
76650,경남인베스트먼트(주),51778,2013/12,68,25.87,2.82,2.82,0.0,-50846.0,3102452.0,...,331478734.0,68119927.0,263358807.0,부동산업,126400000.0,-12332.0,0.0,-1.083535,2013,1.0
81381,(주)코웰메디,53140,2014/12,27,142.92,43.17,81.68,0.0,663730.0,162615.0,...,10387326.0,6111253.0,4276073.0,제조업,3147250.0,110.0,378055.0,-0.358670,2014,1.0
87307,(주)동방헌,56748,2014/12,41,0.00,61.04,61.04,0.0,-1292141.0,1283481.0,...,16207349.0,26550630.0,-10343281.0,건설업,300000.0,-1286.0,0.0,35.477603,2014,1.0
102606,주식회사프라브컴퍼니,61096,2016/12,47,205.69,47.27,105.37,0.0,-142790.0,1193121.0,...,51989984.0,34982658.0,17007326.0,도매 및 소매업,500000.0,5925.0,57469.0,-33.014652,2016,1.0
113734,주식회사씨엔씨이엔지,63993,2020/12,25,557.51,65.45,84.39,0.0,-161115.0,469159.0,...,14649914.0,12421810.0,2228104.0,제조업,450000.0,-1394.0,15800.0,-3.951342,2020,1.0
125979,주식회사광일아이리스,67784,2012/12,68,336641.95,8.53,100.03,0.0,-12749850.0,6100.0,...,13190182.0,13186265.0,3917.0,부동산업,10000.0,0.0,0.0,0.608300,2012,1.0
128596,클레온(주),69204,2013/12,29,0.00,86.84,131.69,0.0,1919539.0,256960.0,...,7617819.0,8625563.0,-1007744.0,제조업,700000.0,-435.0,0.0,2.439634,2013,1.0
135567,(주)제이메탈,71178,2013/12,38,615.57,26.80,53.61,0.0,168717.0,303340.0,...,10288578.0,8850761.0,1437817.0,"수도, 하수 및 폐기물 처리, 원료 재생업",100000.0,-976.0,0.0,-13.378170,2013,1.0
143025,주식회사대양테크닉,74271,2014/12,30,484.58,170.21,194.32,0.0,1381113.0,73177.0,...,12982588.0,10761747.0,2220841.0,제조업,200000.0,231.0,0.0,-10.104205,2014,1.0


In [8]:
df['이자비용1'] = df['영업손익(천원)'] / df['이자보상배율']

In [9]:
df['이자비용1'].value_counts()

-inf             194
 0.000000e+00    188
 inf             116
 1.000000e+00     33
 2.000000e+00     32
                ... 
 1.770990e+05      1
 5.241510e+04      1
 1.164915e+06      1
 1.744823e+06      1
 2.407035e+05      1
Name: 이자비용1, Length: 194542, dtype: int64

---

In [5]:
df = pd.read_csv('../data/t-1_raw_외감.csv', encoding='euc-kr')

In [3]:
# 연도별 이자보상배율의 산업평균, 표준편차 계산
industry_avg_std = df.groupby(['산업분류', 'Year'])['이자보상배율'].agg(['mean', 'std']).reset_index()
industry_avg_std.rename(columns={'mean': '산업평균이자보상배율', 'std': '이자보상배율표준편차'}, inplace=True)

# 산업평균이자보상배율, 이자보상배율표준편차 을 원본 데이터프레임과 merge
df = df.merge(industry_avg_std, on=['산업분류', 'Year'], how='left')

# Function to check if 이자보상배율 is outside the range of 1 이자보상배율표준편차 from the 산업평균이자보상배율
def is_outside_range(row):
    lower_bound = row['산업평균이자보상배율'] - row['이자보상배율표준편차']
    upper_bound = row['산업평균이자보상배율'] + row['이자보상배율표준편차']
    if row['이자보상배율'] < lower_bound or row['이자보상배율'] > upper_bound:
        return 1
    else:
        return 0

# Apply the function to create the '산업평균이자보상배율초과여부' column
df['산업평균이자보상배율초과여부'] = df.apply(is_outside_range, axis=1)


In [ ]:
df['산업분류']

In [6]:
df[['회사명','거래소코드','산업분류','Year','이자보상배율','산업평균이자보상배율','산업평균이자보상배율초과여부','이자보상배율표준편차']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,산업평균이자보상배율,산업평균이자보상배율초과여부,이자보상배율표준편차
0,(주)포스코건설,10000,건설업,2012,1.479000e+01,1.105653e+08,0,3.136891e+08
1,(주)포스코건설,10000,건설업,2013,1.384000e+01,1.145590e+08,0,3.185832e+08
2,(주)포스코건설,10000,건설업,2014,6.330000e+00,1.138165e+08,0,3.176845e+08
3,(주)포스코건설,10000,건설업,2015,7.330000e+00,1.124059e+08,0,3.159564e+08
4,(주)포스코건설,10000,건설업,2016,-8.080000e+00,1.265422e+08,0,3.325476e+08
5,(주)포스코건설,10000,건설업,2017,7.590000e+00,1.345134e+08,0,3.412841e+08
6,(주)포스코건설,10000,건설업,2018,8.240000e+00,1.263785e+08,0,3.323516e+08
7,(주)포스코건설,10000,건설업,2019,1.167000e+01,1.027385e+08,0,3.036405e+08
8,(주)포스코건설,10000,건설업,2020,1.626000e+01,1.177102e+08,0,3.223312e+08
9,(주)포스코건설,10000,건설업,2021,2.369000e+01,1.248882e+08,0,3.305389e+08


In [12]:
import pandas as pd
import numpy as np

# 연도별로 이자보상배율의 산업 평균과 3% 수준의 트리밍 평균을 계산합니다.
def trimmed_mean_3(x):
    trim_pct = 0.03
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

industry_avg = df.groupby(['산업분류', 'Year'])['이자보상배율'].mean().reset_index()
industry_avg.rename(columns={'이자보상배율': '산업평균'}, inplace=True)

industry_avg['트리밍평균 (3%)'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].apply(trimmed_mean_3).reset_index(drop=True)

# 산업 평균과 트리밍 평균을 원본 DataFrame에 다시 병합합니다.
df = df.merge(industry_avg, on=['산업분류', 'Year'], how='left')

# 함수를 사용하여 이자보상배율이 1 표준편차 범위를 벗어나는지 여부를 확인합니다.
def is_outside_range(row):
    lower_bound = row['트리밍평균 (3%)'] - row['이자보상배율'].std()
    upper_bound = row['트리밍평균 (3%)'] + row['이자보상배율'].std()
    if row['이자보상배율'] < lower_bound or row['이자보상배율'] > upper_bound:
        return 1
    else:
        return 0

# '이자보상배율산업평균초과여부' 열을 생성합니다.
df['이자보상배율산업평균초과여부'] = df.apply(is_outside_range, axis=1)


AttributeError: 'float' object has no attribute 'std'

In [11]:
import pandas as pd
import numpy as np

# 연도별로 이자보상배율의 산업 평균과 3% 수준의 트리밍 평균을 계산합니다.
def trimmed_mean_3(x):
    trim_pct = 0.03
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = [val for val in x if lower_bound <= val <= upper_bound]
    return np.mean(trimmed_values)

# 산업 평균과 트리밍 평균을 계산하여 새로운 데이터프레임을 생성합니다.
industry_avg = df.groupby(['산업분류', 'Year'])['이자보상배율'].mean().reset_index()
industry_avg.rename(columns={'이자보상배율': '산업평균'}, inplace=True)

industry_avg['트리밍평균 (3%)'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].apply(trimmed_mean_3).reset_index(drop=True)

# 산업 평균과 트리밍 평균을 원본 DataFrame에 다시 병합합니다.
df = df.merge(industry_avg, on=['산업분류', 'Year'], how='left')

# 함수를 사용하여 이자보상배율이 1 표준편차 범위를 벗어나는지 여부를 확인합니다.
def is_outside_range(row):
    lower_bound = row['트리밍평균 (3%)'] - row['이자보상배율'].std()
    upper_bound = row['트리밍평균 (3%)'] + row['이자보상배율'].std()
    if row['이자보상배율'] < lower_bound or row['이자보상배율'] > upper_bound:
        return 1
    else:
        return 0

# '이자보상배율산업평균초과여부' 열을 생성합니다.
df['이자보상배율산업평균초과여부'] = df.apply(is_outside_range, axis=1)


KeyError: '트리밍평균 (3%)'

---

In [11]:
import pandas as pd
import numpy as np

# 1퍼센트 절사평균 계산 함수
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

# 산업평균 계산
industry_avg = df.groupby(['산업분류', 'Year'])['이자보상배율'].apply(trimmed_mean_1).reset_index()
industry_avg.rename(columns={'이자보상배율': '산업평균'}, inplace=True)

# 산업평균과 원본 DataFrame을 병합
df = df.merge(industry_avg, on=['산업분류', 'Year'], how='left')

# 함수를 사용하여 이자보상배율이 1표준편차 범위를 벗어나는지 여부를 확인
def is_outside_range(row):
    lower_bound = row['산업평균'] - row['이자보상배율'].std()
    upper_bound = row['산업평균'] + row['이자보상배율'].std()
    if row['이자보상배율'] < lower_bound or row['이자보상배율'] > upper_bound:
        return 1
    else:
        return 0

# '이자보상배율산업평균초과여부' 열을 생성
df['이자보상배율산업평균초과여부'] = df.apply(is_outside_range, axis=1)

KeyError: '산업평균'

In [7]:
import pandas as pd
import numpy as np

# 1퍼센트 절사평균 계산 함수
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

df['산업']

# 산업평균 계산
industry_avg = df.groupby(['산업분류', 'Year'])['이자보상배율'].apply(trimmed_mean_1).reset_index()
industry_avg.rename(columns={'이자보상배율': '산업평균'}, inplace=True)

# 산업평균과 원본 DataFrame을 병합
df = df.merge(industry_avg, on=['산업분류', 'Year'], how='left')

# 함수를 사용하여 이자보상배율이 1표준편차 범위를 벗어나는지 여부를 확인
def is_outside_range(row):
    lower_bound = row['산업평균'] - df[(df['산업분류']==row['산업분류'])&(df['Year']==row['Year'])]['이자보상배율'].std()
    upper_bound = row['산업평균'] + df[(df['산업분류']==row['산업분류'])&(df['Year']==row['Year'])]['이자보상배율'].std()
    if row['이자보상배율'] < lower_bound or row['이자보상배율'] > upper_bound:
        return 1
    else:
        return 0

# '이자보상배율산업평균초과여부' 열을 생성
df['이자보상배율산업평균초과여부'] = df.apply(is_outside_range, axis=1)


KeyError: '산업평균'

In [14]:
import pandas as pd
import numpy as np

# 산업분류와 Year를 기준으로 그룹화하여 이자보상배율의 년도별 산업평균을 1% 절사평균으로 구함
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

df['산업년도별이자보상배율절사평균'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)

# 년도별 산업절사평균 이자보상배율에 대한 표준편차를 구함
industry_year_std = df.groupby(['산업분류', 'Year'])['산업년도별이자보상배율절사평균'].std().reset_index()
industry_year_std.rename(columns={'산업년도별이자보상배율절사평균': '이보배절사평편차'}, inplace=True)

# 각 기업에 맞게 '산업년도별이자보상배율절사평균'과 '이보배절사평편차' 컬럼에 값을 입력
df = df.merge(industry_year_std, on=['산업분류', 'Year'], how='left')

In [16]:
df[['회사명','거래소코드','산업분류','Year','이자보상배율', '산업년도별이자보상배율절사평균', '이자보상배율산업평균초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,산업년도별이자보상배율절사평균,이보배절사평편차
0,(주)포스코건설,10000,건설업,2012,1.479000e+01,1.117321e+08,0.0
1,(주)포스코건설,10000,건설업,2013,1.384000e+01,1.157329e+08,0.0
2,(주)포스코건설,10000,건설업,2014,6.330000e+00,1.150065e+08,0.0
3,(주)포스코건설,10000,건설업,2015,7.330000e+00,1.135968e+08,0.0
4,(주)포스코건설,10000,건설업,2016,-8.080000e+00,1.278447e+08,0.0
5,(주)포스코건설,10000,건설업,2017,7.590000e+00,1.359148e+08,0.0
6,(주)포스코건설,10000,건설업,2018,8.240000e+00,1.276619e+08,0.0
7,(주)포스코건설,10000,건설업,2019,1.167000e+01,1.037787e+08,0.0
8,(주)포스코건설,10000,건설업,2020,1.626000e+01,1.189063e+08,0.0
9,(주)포스코건설,10000,건설업,2021,2.369000e+01,1.257558e+08,0.0


---

In [31]:
df = pd.read_csv('../data/t-1_raw_외감.csv', encoding='euc-kr')

In [32]:
df = df[np.isfinite(df['이자보상배율'])]

In [ ]:
import pandas as pd
import numpy as np


# 산업분류와 Year를 기준으로 그룹화하여 이자보상배율의 년도별 산업평균을 1% 절사평균으로 구함
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

df['산업평균이자보상배율'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)

# '이자보상배율'에서 '산업평균이자보상배율'을 빼고, 표준편차를 계산하여 '이자보상배율산업평균초과여부' 컬럼에 1 또는 0을 입력
def is_outside_range(row):
    mean_diff = row['이자보상배율'] - row['산업평균이자보상배율']
    std_dev = df[df['산업분류'] == row['산업분류']][df['Year'] == row['Year']]['이자보상배율'].std()
    if abs(mean_diff) > std_dev:
        return 1
    else:
        return 0

df['이자보상배율산업평균초과여부'] = df.apply(is_outside_range, axis=1)

In [26]:
import pandas as pd
import numpy as np


# 산업분류와 Year를 기준으로 그룹화하여 이자보상배율의 년도별 산업평균을 1% 절사평균으로 구함
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

df['산업평균이자보상배율'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)

# 산업평균이자보상배율의 표준편차 계산
df['이자보상배율표준편차'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform('std')

# 이자보상배율이 표준편차 범위 안에 들어오면 0, 밖으로 벗어나면 1을 입력하는 컬럼 생성
df['이자보상배율산업평균초과여부'] = np.where(
    (df['이자보상배율'] >= df['산업평균이자보상배율'] - df['이자보상배율표준편차']) &
    (df['이자보상배율'] <= df['산업평균이자보상배율'] + df['이자보상배율표준편차']),
    0, 1
)

In [30]:
df[['회사명', '거래소코드', '산업분류', 'Year', '이자보상배율', '산업평균이자보상배율']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,산업평균이자보상배율
0,(주)포스코건설,10000,건설업,2012,1.479000e+01,1.117321e+08
1,(주)포스코건설,10000,건설업,2013,1.384000e+01,1.157329e+08
2,(주)포스코건설,10000,건설업,2014,6.330000e+00,1.150065e+08
3,(주)포스코건설,10000,건설업,2015,7.330000e+00,1.135968e+08
4,(주)포스코건설,10000,건설업,2016,-8.080000e+00,1.278447e+08
5,(주)포스코건설,10000,건설업,2017,7.590000e+00,1.359148e+08
6,(주)포스코건설,10000,건설업,2018,8.240000e+00,1.276619e+08
7,(주)포스코건설,10000,건설업,2019,1.167000e+01,1.037787e+08
8,(주)포스코건설,10000,건설업,2020,1.626000e+01,1.189063e+08
9,(주)포스코건설,10000,건설업,2021,2.369000e+01,1.257558e+08


In [36]:
import pandas as pd
import numpy as np

# 산업평균이자보상배율을 1% 절사평균으로 구함
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

# 산업평균이자보상배율과 표준편차 계산
df['산업평균이자보상배율'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)
df['이자보상배율표준편차'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(lambda x: trimmed_mean_1(x).std())

# 이자보상배율을 표준화하여 표준편차 범위 안에 들어오는지 확인
df['이자보상배율표준화'] = (df['이자보상배율'] - df['산업평균이자보상배율'])

# 표준화된 값이 범위 안에 들어오면 0, 밖이면 1을 입력하는 컬럼 생성
df['이자보상배율산업평균초과여부'] = np.where(
    (df['이자보상배율표준화'] >= -1) & (df['이자보상배율표준화'] <= 1),
    0, 1
)

# 불필요한 컬럼 제거 (선택사항)
df.drop(columns=['이자보상배율표준화'], inplace=True)


AttributeError: 'float' object has no attribute 'std'

---

In [68]:
df = pd.read_csv('../data/t-1_raw_외감_1.csv', encoding='euc-kr')

In [51]:
df.isna().sum()

회사명                   0
거래소코드                 0
회계년도                  0
산업분류코드                0
부채비율                 57
당좌비율                 57
유동비율                 57
이자보상배율               57
OCF(천원)              55
이자비용(천원)             55
차입금의존도               57
자기자본구성비율             57
당기순이익(천원)            55
감가상각비(백만원)         1312
금융비용(백만원)           187
단기차입금(천원)            55
유동성장기부채(천원)          55
장기차입금(천원)            55
법인세비용차감전손익(백만원)      72
매출액영업이익률             57
자기자본순이익률             57
총자본순이익률              57
매출액(천원)              55
총자본회전률               57
자기자본회전률              57
운전자본회전률              57
순운전자본회전률             57
재고자산회전률              57
당좌자산회전률              57
유동자산회전률              57
매출액증가율               56
총자본증가율               56
자기자본증가율              56
순이익증가율               56
유형자산증가율              56
유동자산증가율              56
재고자산증가율              56
영업이익증가율              56
총자본투자효율              66
부가가치율                66
노동소득분배율              66
자본분배율           

In [47]:
import pandas as pd
import numpy as np

# 산업평균이자보상배율을 1% 절사평균으로 구함
def trimmed_mean_1(x):
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

# 산업평균이자보상배율과 표준편차 계산
df['산업평균이자보상배율'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)
df['이자보상배율표준편차'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(lambda x: trimmed_mean_1(x).std())

# 이자보상배율을 표준화하여 표준편차 범위 안에 들어오는지 확인
df['이자보상배율표준화'] = (df['이자보상배율'] - df['산업평균이자보상배율'])

# 표준화된 값이 범위 안에 들어오면 0, 밖이면 1을 입력하는 컬럼 생성
df['이자보상배율산업평균초과여부'] = np.where(
    (df['이자보상배율표준화'] >= -1) & (df['이자보상배율표준화'] <= 1),
    0, 1
)

# 불필요한 컬럼 제거 (선택사항)
df.drop(columns=['이자보상배율표준화'], inplace=True)

AttributeError: 'float' object has no attribute 'std'

In [ ]:
# Define the trimmed mean function with check for empty data
def trimmed_mean_1(x):
    if len(x) == 0:  # check if the data is empty
        return np.nan
    trim_pct = 0.01
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()
# Compute the trimmed mean for each industry and year
df['산업평균이자보상배율'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(lambda x: trimmed_mean_1(x))
# Compute the standard deviation for trimmed values for each industry and year
df['이자보상배율표준편차'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(
    lambda x: np.std(x[(x >= np.percentile(x, 0.01 * 100)) & (x <= np.percentile(x, (1 - 0.01) * 100))]) if len(x) > 0 else np.nan
)
# Standardize the '이자보상배율'
df['이자보상배율표준화'] = (df['이자보상배율'] - df['산업평균이자보상배율'])
# Check if the standardized value is within the standard deviation range
df['이자보상배율산업평균초과여부'] = np.where(
    (df['이자보상배율표준화'] >= -df['이자보상배율표준편차']) & (df['이자보상배율표준화'] <= df['이자보상배율표준편차']),
    0, 1
)

In [52]:
df.columnsq

Index(['회사명', '거래소코드', '회계년도', '산업분류코드', '부채비율', '당좌비율', '유동비율', '이자보상배율',
       'OCF(천원)', '이자비용(천원)', '차입금의존도', '자기자본구성비율', '당기순이익(천원)', '감가상각비(백만원)',
       '금융비용(백만원)', '단기차입금(천원)', '유동성장기부채(천원)', '장기차입금(천원)', '법인세비용차감전손익(백만원)',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '매출액(천원)', '총자본회전률', '자기자본회전률',
       '운전자본회전률', '순운전자본회전률', '재고자산회전률', '당좌자산회전률', '유동자산회전률', '매출액증가율',
       '총자본증가율', '자기자본증가율', '순이익증가율', '유형자산증가율', '유동자산증가율', '재고자산증가율',
       '영업이익증가율', '총자본투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', 'ICF(천원)',
       'FCF(천원)', '자산총계(천원)', '부채총계(천원)', '자본총계(천원)', '산업분류', '자본금(천원)',
       '영업손익(천원)', '연구개발비(천원)', '자본잠식률', 'Year', 't-1감사의견코드'],
      dtype='object')

In [62]:
df.describe()

,거래소코드,산업분류코드,부채비율,당좌비율,유동비율,이자보상배율,OCF(천원),이자비용(천원),차입금의존도,자기자본구성비율,...,자본금(천원),영업손익(천원),연구개발비(천원),자본잠식률,Year,t-1감사의견코드,평균이보배,이보배표준편차,초과이보배,이보배초과여부
count,188707.000000,188707.000000,1.886500e+05,1.886500e+05,1.886500e+05,1.886500e+05,1.886520e+05,1.886520e+05,188650.000000,188650.000000,...,1.886520e+05,1.886520e+05,1.886520e+05,188644.000000,188707.000000,188707.000000,1.887060e+05,1.887060e+05,1.886500e+05,188707.000000
mean,57672.474339,44.055255,1.797187e+04,4.346751e+03,5.463465e+03,1.264920e+08,3.878672e+06,1.009580e+06,40.997030,38.188565,...,6.701427e+06,3.683186e+06,3.687426e+05,-38.063035,2016.871891,0.071460,1.265148e+08,3.201895e+08,0.000000e+00,0.873301
std,25367.516420,20.438501,5.507996e+06,6.081015e+05,6.800613e+05,3.323922e+08,7.543658e+07,6.953223e+06,238.396729,27.920673,...,6.134369e+07,3.965322e+07,1.468650e+07,473.441759,2.866913,0.257592,6.417310e+07,6.214138e+07,3.261451e+08,0.332636
min,10000.000000,1.000000,-6.280000e+00,-5.400000e-01,-5.400000e-01,-3.620049e+07,-1.467355e+09,0.000000e+00,0.000000,-22.870000,...,0.000000e+00,-1.936136e+09,-1.904800e+05,-97251.240000,2012.000000,0.000000,1.100000e-01,0.000000e+00,-6.000001e+08,0.000000
25%,41840.000000,26.000000,4.163250e+01,3.659000e+01,6.522000e+01,3.800000e-01,-3.720290e+05,4.756850e+04,10.160000,14.490000,...,4.000000e+05,-2.331375e+04,0.000000e+00,-19.395381,2014.000000,0.000000,7.783116e+07,2.677984e+08,-1.437206e+08,1.000000
50%,59716.000000,42.000000,1.363750e+02,8.642500e+01,1.195600e+02,2.770000e+00,7.989190e+05,2.358080e+05,37.720000,34.910000,...,1.000000e+06,7.167140e+05,0.000000e+00,-5.453628,2017.000000,0.000000,1.159135e+08,3.201212e+08,-8.367652e+07,1.000000
75%,76684.000000,68.000000,3.461875e+02,1.721500e+02,2.212975e+02,2.497750e+01,2.978970e+06,5.835622e+05,62.360000,59.190000,...,3.000000e+06,2.344441e+06,7.200000e+02,-0.486177,2019.000000,0.000000,1.518576e+08,3.588825e+08,-7.211777e+07,1.000000
max,107278.000000,98.000000,2.373601e+09,1.715128e+08,1.715128e+08,1.733808e+09,1.094303e+10,5.937832e+08,102094.630000,106.700000,...,7.688422e+09,5.268395e+09,2.585544e+09,13323.089000,2022.000000,1.000000,1.000000e+09,5.000000e+08,1.652643e+09,1.000000


In [88]:
df.std()

거래소코드         2.511817e+04
Year          2.868162e+00
산업분류코드        2.031295e+01
부채비율          5.302300e+05
당좌비율          7.365654e+04
                  ...     
평균부채비율초과여부    4.998856e-01
t-1감사의견코드     2.547134e-01
평균이보배         6.957193e+07
이보배표준편차       7.634080e+07
이보배초과여부       4.913891e-01
Length: 68, dtype: float64

In [84]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/train_data_t-1.csv', encoding='euc-kr')

# 데이터프레임에서 '이자보상배율' 열의 1퍼센트 절사평균과 표준편차 계산
def trimmed_mean_1(x):
    trim_pct = 0.05
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.mean()

def trimmed_std_1(x):
    trim_pct = 0.05
    lower_bound = np.percentile(x, trim_pct * 100)
    upper_bound = np.percentile(x, (1 - trim_pct) * 100)
    trimmed_values = x[(x >= lower_bound) & (x <= upper_bound)]
    return trimmed_values.std()

df['평균이보배'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_mean_1)
df['이보배표준편차'] = df.groupby(['산업분류', 'Year'])['이자보상배율'].transform(trimmed_std_1)

# 개별 이자보상배율과 평균-표준편차를 비교하여 이보배초과여부 계산
df['이보배초과여부'] = np.where(df['이자보상배율'] < (df['평균이보배'] - 0.3*df['이보배표준편차']), 1, 0)


In [85]:
df[['회사명','거래소코드','산업분류','Year','이자보상배율', '평균이보배', '이보배초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,이자보상배율,평균이보배,이보배초과여부
0,주식회사한스갤러리,78935,제조업,2017,-4.000000e+00,8.781668e+07,1
1,주식회사해냄파트너스,63254,부동산업,2015,2.160000e+00,1.369614e+08,1
2,현대종합특수강주식회사,60665,제조업,2012,9.300000e-01,5.516769e+07,0
3,주식회사덕성하이텍,17699,제조업,2019,1.149000e+01,8.600241e+07,1
4,(주)위동해운,56339,운수 및 창고업,2014,5.461000e+01,1.021900e+08,1
5,주식회사용천인터내셔날,72784,제조업,2014,1.540000e+00,6.604836e+07,0
6,비아이디씨주식회사,52060,운수 및 창고업,2015,4.679000e+01,9.824604e+07,1
7,(주)삼라마이다스,61412,도매 및 소매업,2021,-4.100000e-01,1.518822e+08,1
8,주식회사스타메딕스,74051,도매 및 소매업,2020,1.390000e+00,1.422232e+08,1
9,(주)한일엠이씨,48330,"전문, 과학 및 기술 서비스업",2013,1.000000e+09,3.322621e+08,0


In [86]:
df['이보배초과여부'].value_counts()

1    81561
0    56117
Name: 이보배초과여부, dtype: int64

In [87]:
df[df['이보배초과여부']==1]['t-1감사의견코드'].value_counts()

0.0    76202
1.0     5359
Name: t-1감사의견코드, dtype: int64

In [65]:
df[['회사명','거래소코드','산업분류','Year','부채비율', '평균부비', '부비초과여부']].head(50)

,회사명,거래소코드,산업분류,Year,부채비율,평균부비,부비초과여부
0,(주)포스코건설,10000,건설업,2012,173.49,2550.640813,1
1,(주)포스코건설,10000,건설업,2013,130.89,955.422383,1
2,(주)포스코건설,10000,건설업,2014,120.92,1521.520824,1
3,(주)포스코건설,10000,건설업,2015,100.59,1248.140441,1
4,(주)포스코건설,10000,건설업,2016,145.02,1249.760191,1
5,(주)포스코건설,10000,건설업,2017,150.38,1275.229005,1
6,(주)포스코건설,10000,건설업,2018,119.29,828.732749,1
7,(주)포스코건설,10000,건설업,2019,122.59,1314.457444,1
8,(주)포스코건설,10000,건설업,2020,113.56,5162.345237,1
9,(주)포스코건설,10000,건설업,2021,113.09,9600.559456,1
